In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
import os
from src.pcai_datamodule import PatchDataModule
from src.pcai_litmodule import LitModuleClasAdversarial

In [4]:
dataset_dir = '../data/tma_dataset/'
image_base_dir = os.path.join(dataset_dir, "images")
mask_base_dir = os.path.join(dataset_dir, "masks/masks/")
metadata_path = os.path.join(dataset_dir, 'metadata/metadata.csv')

In [5]:
datamodule = PatchDataModule(
    metadata_path=metadata_path,
    image_base_dir=image_base_dir,
    mask_base_dir=mask_base_dir,  
    label_cols_clas=['event_duration', 'event_indicator'],
    label_col_domain=['domain'],
    patch_size=14,
    n_patches=5, # return 5 patches per bag
)

datamodule.setup("fit")

In [6]:
sample = next(iter(datamodule.train_dataloader()))
len(sample)

4

In [7]:
sample[0].shape

torch.Size([2, 5, 3, 14, 14])

In [8]:
litmodule = LitModuleClasAdversarial()

In [9]:
from pytorch_lightning import Trainer

In [10]:
trainer = Trainer(accelerator="cpu", max_epochs=1)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [11]:
trainer.fit(model=litmodule, datamodule=datamodule)


  | Name           | Type             | Params
----------------------------------------------------
0 | net            | AdversarialNet   | 6.5 M 
1 | criterion_clas | CrossEntropyLoss | 0     
2 | criterion_adv  | CrossEntropyLoss | 0     
----------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
25.913    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1558: PossibleUserWarning: The number of training batches (1) is sm

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [12]:
predictions = trainer.predict(model=litmodule, datamodule=datamodule)
predictions

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 1it [00:00, ?it/s]

[{'x': {'clas': tensor([[-0.0097, -0.0775]]),
   'adv': tensor([[-0.1080, -0.0983,  0.0352]])},
  'y': {'clas': tensor([0]), 'adv': tensor([0])}},
 {'x': {'clas': tensor([[-0.0097, -0.0775]]),
   'adv': tensor([[-0.1080, -0.0983,  0.0352]])},
  'y': {'clas': tensor([1]), 'adv': tensor([1])}},
 {'x': {'clas': tensor([[-0.0097, -0.0775]]),
   'adv': tensor([[-0.1080, -0.0983,  0.0352]])},
  'y': {'clas': tensor([1]), 'adv': tensor([2])}}]